In [231]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder


rn_state = 42 # random state for reproducibility

**1. Analisis del dataset** (Analisis de datos, limpieza y preparacion)

In [232]:
df = pd.read_csv('./Sleep_health_and_lifestyle_dataset.csv')
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [233]:
# Obtener información sobre el conjunto de datos
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB
None


In [234]:
# Resumen estadístico de las variables numéricas
print(df.describe())

        Person ID         Age  Sleep Duration  Quality of Sleep  \
count  374.000000  374.000000      374.000000        374.000000   
mean   187.500000   42.184492        7.132086          7.312834   
std    108.108742    8.673133        0.795657          1.196956   
min      1.000000   27.000000        5.800000          4.000000   
25%     94.250000   35.250000        6.400000          6.000000   
50%    187.500000   43.000000        7.200000          7.000000   
75%    280.750000   50.000000        7.800000          8.000000   
max    374.000000   59.000000        8.500000          9.000000   

       Physical Activity Level  Stress Level  Heart Rate   Daily Steps  
count               374.000000    374.000000  374.000000    374.000000  
mean                 59.171123      5.385027   70.165775   6816.844920  
std                  20.830804      1.774526    4.135676   1617.915679  
min                  30.000000      3.000000   65.000000   3000.000000  
25%                  45.000000 

In [235]:
# Verificar si hay valores faltantes
print(df.isnull().sum())

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64


In [236]:
# Manejo de valores faltantes en 'Sleep Disorder
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

# Elimino los datos que no aportan valor
df.drop(columns=['Person ID'], inplace=True)
df_clean = df.dropna()

In [237]:
# Verificar valores únicos en columnas categóricas
print(df_clean['Gender'].unique())
print(df_clean['Occupation'].unique())
print(df_clean['BMI Category'].unique())
print(df_clean['Sleep Disorder'].unique())

['Male' 'Female']
['Software Engineer' 'Doctor' 'Sales Representative' 'Teacher' 'Nurse'
 'Engineer' 'Accountant' 'Scientist' 'Lawyer' 'Salesperson' 'Manager']
['Overweight' 'Normal' 'Obese' 'Normal Weight']
['None' 'Sleep Apnea' 'Insomnia']


In [238]:
# Dividir la columna 'Blood Pressure' en presión sistólica y diastólica
df_clean[['Systolic Pressure', 'Diastolic Pressure']] = df_clean['Blood Pressure'].str.split('/', expand=True)

# Convertir las nuevas columnas a tipo numérico
df_clean['Systolic Pressure'] = pd.to_numeric(df_clean['Systolic Pressure'])
df_clean['Diastolic Pressure'] = pd.to_numeric(df_clean['Diastolic Pressure'])

# Eliminar la columna original 'Blood Pressure'
df_clean.drop('Blood Pressure', axis=1, inplace=True)

In [239]:
categorical_columns = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']

# Iterar sobre cada columna categórica y aplicar el Label Encoding
label_encoder = LabelEncoder()
for column in categorical_columns:
    df_clean[column] = label_encoder.fit_transform(df_clean[column])


# Verificar los cambios
print(df_clean.head())

   Gender  Age  Occupation  Sleep Duration  Quality of Sleep  \
0       1   27           9             6.1                 6   
1       1   28           1             6.2                 6   
2       1   28           1             6.2                 6   
3       1   28           6             5.9                 4   
4       1   28           6             5.9                 4   

   Physical Activity Level  Stress Level  BMI Category  Heart Rate  \
0                       42             6             3          77   
1                       60             8             0          75   
2                       60             8             0          75   
3                       30             8             2          85   
4                       30             8             2          85   

   Daily Steps  Sleep Disorder  Systolic Pressure  Diastolic Pressure  
0         4200               1                126                  83  
1        10000               1                125 

**2. Separamos los datos en test y entrenamiento**

In [240]:
# Dividir el conjunto de datos en características (X) y la variable objetivo (y)
X = df_clean.drop('Stress Level', axis=1)  # Características
y = df_clean['Stress Level']  # Variable objetivo

# Dividir los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento, 20% para prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos de datos resultantes
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)


Forma de X_train: (299, 12)
Forma de X_test: (75, 12)
Forma de y_train: (299,)
Forma de y_test: (75,)


**3. Creamos el modelo**

In [241]:
# Inicializar el modelo de Bosque Aleatorio
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo
random_forest.fit(X_train, y_train)

# Predecir el nivel de estrés en el conjunto de prueba
y_pred = random_forest.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

Error cuadrático medio (MSE): 0.03074399999999999
Coeficiente de determinación (R^2): 0.9901596107886651


**4. Probamos el modelo**

In [242]:
# Seleccionar un ejemplo del conjunto de datos
example_index = 5  # Por ejemplo, seleccionamos el primer ejemplo
example = X_test.iloc[[example_index]]  # Seleccionamos el ejemplo del conjunto de prueba

# Predecir el nivel de estrés para el ejemplo seleccionado
predicted_stress_level = random_forest.predict(example)

# Imprimir la predicción
print("El nivel de estrés predicho para el ejemplo seleccionado es:", predicted_stress_level[0])


El nivel de estrés predicho para el ejemplo seleccionado es: 7.0


**5. Determinamos que variables aportan son mas importantes para predecir el nivel de estres**

In [243]:
# Obtener la importancia de las características del modelo de Bosque Aleatorio
feature_importances = random_forest.feature_importances_

# Crear un DataFrame para visualizar las importancias de las características
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Imprimir las características más importantes
print("Variables más importantes para predecir el nivel de estrés:")
print(feature_importance_df)


Variables más importantes para predecir el nivel de estrés:
                    Feature  Importance
4          Quality of Sleep    0.734901
7                Heart Rate    0.123631
3            Sleep Duration    0.074013
8               Daily Steps    0.016375
2                Occupation    0.014493
5   Physical Activity Level    0.012852
1                       Age    0.010325
0                    Gender    0.006837
10        Systolic Pressure    0.002598
11       Diastolic Pressure    0.001875
6              BMI Category    0.001213
9            Sleep Disorder    0.000888


**6. Clasificamos los resultados**

In [244]:
# Definir la función para clasificar las predicciones en las categorías deseadas
def classify_stress_level(prediction):
    if prediction <= 3:
        return "No estresado"
    elif prediction <= 6:
        return "Estresado"
    else:
        return "Extremadamente estresado"

# Aplicar la función de clasificación a las predicciones del modelo
classified_predictions = [classify_stress_level(pred) for pred in y_pred]

# Imprimir las predicciones clasificadas
print("Resultados del algoritmo clasificados en las 3 categorías:")
print(classified_predictions)


Resultados del algoritmo clasificados en las 3 categorías:
['No estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'No estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Estresado', 'No estresado', 'Estresado', 'Extremadamente estresado', 'Estresado', 'Estresado', 'Extremadamente estresado', 'Estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'No estresado', 'Estresado', 'Estresado', 'Estresado', 'Extremadamente estresado', 'Estresado', 'Estresado', 'Estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'No estresado', 'Estresado', 'Estresado', 'Estresado', 'No estresado', 'Extremadamente estresado', 'Estresado', 'Extremadamente estresado', 'Estresado', 'Estresado', 'Estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Extremadamente estresado', 'Estresado', 'Estresado', 'Estresado', 'Estresado'

**7. Conclusiones**

En conclusión, si bien el conjunto de datos proporcionado incluye una variedad de características relevantes, es probable que no capture toda la complejidad del fenómeno del estrés. Por lo tanto, aunque el algoritmo podría proporcionar cierta capacidad para detectar el estrés de las personas, es posible que no sea completamente preciso o generalizable en todas las situaciones. Sería beneficioso considerar la adición de más características relevantes y la recopilación de datos adicionales para mejorar la capacidad del algoritmo para detectar el estrés de manera más precisa.